In [2]:
%pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl (2.9 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706
Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup
import streamlit as st
import re
import pandas as pd
import requests
import pdfplumber
from datetime import datetime
INDUSTRY_PERCENTILE_FILE = 'datafiles/Akaushi_Percentile_Ranks_LATEST.pdf'
AA_URL = 'https://akaushi.digitalbeef.com/'

In [2]:
#open pickle file with dataframe
import pickle
import pandas as pd
with open('../datafiles/filtered_data.pkl', 'rb') as f:
    df = pickle.load(f)
pd.set_option('display.max_columns', None)

C:\Users\case4\AppData\Local\Temp\ipykernel_41032\1571859427.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(f)


In [ ]:
df.columns.to_list()

In [17]:
import pdfplumber
import pandas as pd
import re

def extract_cattle_statistics(pdf_path):
    """
    Extract cattle statistics tables from PDF with specific column and row patterns.
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        dict: Dictionary of DataFrames, one for each table found
    """
    # Define expected columns
    expected_columns = ['categories', 'CED', 'BW', 'WW', 'YW', 'MK', 'TM', 'Growth']
    
    # Initialize results dictionary
    tables_dict = {}
    
    def is_percentage_row(text):
        """Check if a row starts with a percentage pattern"""
        return bool(re.match(r'^\d+%$', text))
    
    def get_row_values(table_row):
        """Extract numerical values from a row"""
        return [cell.strip() if cell else '' for cell in table_row]
    
    def validate_row_pattern(first_cell):
        """Check if the row follows expected patterns"""
        if first_cell in ['Num Animals', 'High', 'Average', 'Low']:
            return True
        return is_percentage_row(first_cell)
    
    with pdfplumber.open(pdf_path) as pdf:
        current_table = None
        current_data = []
        
        for page in pdf.pages:
            # Extract text and split into lines
            text = page.extract_text()
            lines = text.split('\n')
            
            for line in lines:
                # Skip empty lines
                if not line.strip():
                    continue
                
                # Check for new table header (contains "Statistical Breakdown")
                if "Statistical Breakdown" in line:
                    # Save previous table if it exists
                    if current_data:
                        df = pd.DataFrame(current_data, columns=expected_columns)
                        tables_dict[current_table] = df
                    
                    # Start new table
                    current_table = line.strip()
                    current_data = []
                    continue
                
                # Split line into values
                values = line.split()
                
                # Check if this is a valid data row
                if values and validate_row_pattern(values[0]):
                    # Handle different number formats and clean data
                    row_data = []
                    current_value = ''
                    
                    for val in values:
                        # Add to current_value if it's part of a number
                        if re.match(r'^-?\d*\.?\d*$', val) or val.endswith('%'):
                            if current_value:
                                current_value += ' ' + val
                            else:
                                current_value = val
                        else:
                            if current_value:
                                row_data.append(current_value)
                                current_value = ''
                            row_data.append(val)
                    
                    # Add any remaining value
                    if current_value:
                        row_data.append(current_value)
                    
                    # Only process rows that match our expected pattern
                    if len(row_data) == len(expected_columns):
                        current_data.append(row_data)
        
        # Save the last table
        if current_data:
            df = pd.DataFrame(current_data, columns=expected_columns)
            display(df)
            tables_dict[current_table] = df
        else: 
            print("No tables found in the PDF.")
    
    return tables_dict



# Example usage
if __name__ == "__main__":
    pdf_path = "../datafiles/Akaushi_Percentile_Ranks_LATEST.pdf"
    tables = extract_cattle_statistics(pdf_path)
   

No tables found in the PDF.


In [20]:
activeSiresPercentileRankDf, activeDamsPercentileRankDf, nonParentsPercentileRankDf = extract_IndustryPercentileRankTables_from_pdf()


AttributeError: module 'camelot' has no attribute 'read_pdf'